# Tutorial: Feed Unit Model with Modular Property Package

![](feed.svg)

## Learning Outcomes

- Demonstrate use of the Feed unit model in IDAES


## Problem Statement

In this tutorial, we will be feeding a mixed stream of liquid benzene and liquid toluene. This unit model has no inlets and only one outlet, so the stream information is fed to the model through a feed port rather than an inlet. The feed conditions are as follows:

Flow Rate = 100 mol/s

Mole Fraction (Benzene) = 0.6

Mole Fraction (Toluene) = 0.4

Temperature = 298 K

Pressure = 101325 Pa

For more details, please refer to the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable

## Importing necessary tools

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [ ]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, value

# Import the solver
from idaes.core.solvers import get_solver

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the feed unit model
from idaes.models.unit_models import Feed

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the modular property package to create a property block for the flowsheet
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock

# Import the BT_Ideal property package to create a configuration file for the GenericParameterBlock
from idaes.models.properties.modular_properties.examples.BT_ideal import configuration

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

## Setting up the flowsheet
In the following cell, we will create the `ConcreteModel` foundation, attach the steady state flowsheet, and declare the property parameter block that will used.

More information on this general workflow can be found here: https://idaes-pse.readthedocs.io/en/stable/how_to_guides/workflow/general.html

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False) # dynamic or ss flowsheet needs to be specified here

m.fs.properties = GenericParameterBlock(**configuration)

In the following cell, we will be creating the feed unit model, assigning a property package to it, and determining the initial degrees of freedom associated with the feed unit model.

In [ ]:
m.fs.feed = Feed(property_package=m.fs.properties)

DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom are: {0}'.format(DOF_initial))

In [ ]:
assert DOF_initial == 5

### Fixing input specifications
In the following cell, we will be specifying the conditions for the feed block using a feed port and re-evaluating the degrees of freedom to ensure the problem is square (i.e. DOF=0). While most other unit models simply use the inlet port to fix the unit model's operating conditions, the feed unit model cannot because there is no inlet stream to the feed block. 

In [ ]:
m.fs.feed.flow_mol.fix(100) # converting to mol/s as unit basis is mol/s
m.fs.feed.mole_frac_comp[0, "benzene"].fix(0.6)
m.fs.feed.mole_frac_comp[0, "toluene"].fix(0.4)
m.fs.feed.pressure.fix(101325) # Pa
m.fs.feed.temperature.fix(298) # K

DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

In [ ]:
assert DOF_final == 0

### Flowsheet Initialization

IDAES includes pre-written initialization routines for all unit models. Failing to initialize or having a poor intialization of a flowsheet may result in the problem being unsolvable. The output from initialization can be set to 7 different levels depending on the details required by the user. In general, when a particular output level is set, any information at that level and above gets picked up by logger. The default level taken by the logger is INFO.

More information on these levels can be found in the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable/reference_guides/logging.html

In [ ]:
m.fs.feed.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results
In the following cell, the flowsheet will be solved using the IDAES `get_solver` tool. Setting `tee=True` will display the solver output.

In [ ]:
solver = get_solver()
result = solver.solve(m, tee=True)

In [ ]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results

As expected, the output conditions shown in the report below match the feed conditions that were set earlier.

In [ ]:
m.fs.feed.report()

In [ ]:
import pytest

# Check results
assert value(m.fs.feed.outlet.flow_mol[0]) == pytest.approx(100, rel=1e-6)
assert value(m.fs.feed.mole_frac_comp[0, "benzene"]) == pytest.approx(0.6, rel=1e-6)
assert value(m.fs.feed.mole_frac_comp[0, "toluene"]) == pytest.approx(0.4, rel=1e-6)